In [ ]:
self.CURRENT_ANNOATAION_FLAGS = {"traj" : False  ,
                                        "bbox" : True  ,         
                                        "id" : True ,
                                        "class" : True,
                                        "mask" : True,
                                        "polygons" : True}
self.CURRENT_ANNOATAION_TRAJECTORIES = {}



def openVideo(self):
        self.CURRENT_ANNOATAION_TRAJECTORIES = {}
        
        
        
        
        
self.traj_checkBox.setChecked(False)






def draw_trajectories(self ,img, shapes):
    for shape in shapes:
        id = shape["group_id"]
        pts = self.CURRENT_ANNOATAION_TRAJECTORIES['id_'+str(id)]
        color = self.CURRENT_ANNOATAION_TRAJECTORIES['id_color_'+str(id)]
        for i in range(1, len(pts)):
            thickness = 2
            thickness = int(np.sqrt(30 / float(i + 1)) * 2)
            if pts[i - 1] is None or pts[i] is None:
                continue
            cv2.line(img, pts[i - 1], pts[i], color, thickness)
                    
    return img
    
    
    
    
    
            if self.CURRENT_ANNOATAION_FLAGS['traj']:   
                center = ( int((x1 + x2) / 2), int((y1 + y2) / 2) )
                try:
                    self.CURRENT_ANNOATAION_TRAJECTORIES['id_'+str(id)].appendleft(center)
                except:
                    self.CURRENT_ANNOATAION_TRAJECTORIES['id_'+str(id)] = collections.deque(maxlen = 30)
                    self.CURRENT_ANNOATAION_TRAJECTORIES['id_'+str(id)].appendleft(center)
                    self.CURRENT_ANNOATAION_TRAJECTORIES['id_color_'+str(id)] = color
        
        if self.CURRENT_ANNOATAION_FLAGS['traj']:   
            img = self.draw_trajectories(img, shapes)
        
        qimage = self.convert_cv_to_qt(img, )
        return qimage
    



In [ ]:
def draw_trajectories(self ,img, shapes):
        for shape in shapes:
            id = shape["group_id"]
            pts = self.CURRENT_ANNOATAION_TRAJECTORIES['id_'+str(id)]
            color = self.CURRENT_ANNOATAION_TRAJECTORIES['id_color_'+str(id)]
            for i in range(1, len(pts)):
                thickness = 2
                thickness = int(np.sqrt(30 / float(i + 1)) * 2)
                if pts[i - 1] is None or pts[i] is None:
                    continue
                cv2.line(img, pts[i - 1], pts[i], color, thickness)
                    
        return img
    
    
    
    
    
    
    
    if self.CURRENT_ANNOATAION_FLAGS['traj']:   
                center = ( int((x1 + x2) / 2), int((y1 + y2) / 2) )
                try:
                    self.CURRENT_ANNOATAION_TRAJECTORIES['id_'+str(id)].appendleft(center)
                except:
                    self.CURRENT_ANNOATAION_TRAJECTORIES['id_'+str(id)] = collections.deque(maxlen = 30)
                    self.CURRENT_ANNOATAION_TRAJECTORIES['id_'+str(id)].appendleft(center)
                    self.CURRENT_ANNOATAION_TRAJECTORIES['id_color_'+str(id)] = color

In [23]:
x = {}
y = 7.5
id = '5'
center = (5, -2)

x['id_'+str(id)] = [(-1, - 1)] * int(y)
x['id_'+str(id)][1 - 1] = center
x['id_'+str(id)][7 - 1] = center
x['id_'+str(id)][6 - 1] = center
x

{'id_5': [(5, -2), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (5, -2), (5, -2)]}

In [24]:
pts = x['id_'+str(id)][max(7 - 30, 0) : 7] 
print(pts)
for i in range(len(pts) - 1, 0, - 1):
    print(i)
    if pts[i - 1] is None or pts[i] is None :
        print('here') 
    if pts[i]==(-1, - 1) :
        break

[(5, -2), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (5, -2), (5, -2)]
6
5
4


In [17]:
import numpy as np
p = [1, 2, 3, 4, 5, 6, 7, 8, 2, 3, 4, 5, 6, 7, 8, 9, -1, -2]
points =np.array( [ [x, y] for x, y in zip(p[0::2], p[1::2]) ])
print(points)

[[ 1  2]
 [ 3  4]
 [ 5  6]
 [ 7  8]
 [ 2  3]
 [ 4  5]
 [ 6  7]
 [ 8  9]
 [-1 -2]]


In [21]:
color = (0, 255, 0)
color = tuple(int(0.9 * x) for x in color)
color

(0, 229, 0)

In [40]:
def convert(text):
    if (text[0] > 'a' and text[0] < 'z') or (text[0] > 'A' and text[0] < 'Z'):
        text = text.replace('q', 'ض')
        text = text.replace('w', 'ص')
        text = text.replace('e', 'ث')
        text = text.replace('r', 'ق')
        text = text.replace('t', 'ف')
        text = text.replace('y', 'غ')
        text = text.replace('u', 'ع')
        text = text.replace('i', 'ه')
        text = text.replace('o', 'خ')
        text = text.replace('p', 'ح')
        text = text.replace('[', 'ج')
        text = text.replace(']', 'د')
        text = text.replace('a', 'ش')
        text = text.replace('s', 'س')
        text = text.replace('d', 'ي')
        text = text.replace('f', 'ب')
        text = text.replace('g', 'ل')
        text = text.replace('h', 'ا')
        text = text.replace('j', 'ت')
        text = text.replace('k', 'ن')
        text = text.replace('l', 'م')
        text = text.replace(';', 'ك')
        text = text.replace("'", 'ط')
        text = text.replace('z', 'ئ')
        text = text.replace('x', 'ء')
        text = text.replace('c', 'ؤ')
        text = text.replace('v', 'ر')
        text = text.replace('b', 'لا')
        text = text.replace('n', 'ى')
        text = text.replace('m', 'ة')
        text = text.replace(',', 'و')
        text = text.replace('.', 'ز')
        text = text.replace('/', 'ظ')
        text = text.replace('Q', ' َ')
        text = text.replace('W', ' ً')
        text = text.replace('E', ' ُ')
        text = text.replace('R', ' ٌ')
        text = text.replace('T', ' ْ')
        text = text.replace('Y', ' ّ')
        text = text.replace('U', ' ٍ')
        text = text.replace('I', ' ـ')
        text = text.replace('O', ' ِ')
        text = text.replace('P', ' ٓ')
        text = text.replace('{', ' ٔ')
        text = text.replace('}', ' ٕ')
        text = text.replace('A', ' ٖ')
        text = text.replace('S', ' ٗ')
        text = text.replace('D', ' ٘')
        text = text.replace('F', ' ٙ')
        text = text.replace('G', ' ٚ')
        text = text.replace('H', ' ٛ')
        text = text.replace('J', ' ٜ')
        text = text.replace('K', ' ٝ')
        text = text.replace('L', ' ٞ')
        text = text.replace(':', ' ٟ')
        text = text.replace('"', ' ٠')
        text = text.replace('Z', ' ١')
        text = text.replace('X', ' ٢')
        text = text.replace('C', ' ٣')
        text = text.replace('V', ' ٤')
        text = text.replace('B', ' ٥')
        text = text.replace('N', ' ٦')
        text = text.replace('M', ' ٧')
        text = text.replace('<', ' ٨')
        text = text.replace('>', ' ٩')
        text = text.replace('?', ' ٪')
        
    else:
        text = text.replace('ض', 'q')
        text = text.replace('ص', 'w')
        text = text.replace('ث', 'e')
        text = text.replace('ق', 'r')
        text = text.replace('ف', 't')
        text = text.replace('غ', 'y')
        text = text.replace('ع', 'u')
        text = text.replace('ه', 'i')
        text = text.replace('خ', 'o')
        text = text.replace('ح', 'p')
        text = text.replace('ج', '[')
        text = text.replace('د', ']')
        text = text.replace('ش', 'a')
        text = text.replace('س', 's')
        text = text.replace('ي', 'd')
        text = text.replace('ب', 'f')
        text = text.replace('ل', 'g')
        text = text.replace('ا', 'h')
        text = text.replace('ت', 'j')
        text = text.replace('ن', 'k')
        text = text.replace('م', 'l')
        text = text.replace('ك', ';')
        text = text.replace('ط', "'")
        text = text.replace('ئ', 'z')
        text = text.replace('ء', 'x')
        text = text.replace('ؤ', 'c')
        text = text.replace('ر', 'v')
        text = text.replace('لا', 'b')
        text = text.replace('ى', 'n')
        text = text.replace('ة', 'm')
        text = text.replace('و', ',')
        text = text.replace('ز', '.')
        text = text.replace('ظ', '/')
        text = text.replace(' َ', 'Q')
        text = text.replace(' ً', 'W')
        text = text.replace(' ُ', 'E')
        text = text.replace(' ٌ', 'R')
        text = text.replace(' ْ', 'T')
        text = text.replace(' ّ', 'Y')
        text = text.replace(' ٍ', 'U')
        text = text.replace(' ـ', 'I')
        text = text.replace(' ِ', 'O')
        text = text.replace(' ٓ', 'P')
        text = text.replace(' ٔ', '{')
        text = text.replace(' ٕ', '}')
        text = text.replace(' ٖ', 'A')
        text = text.replace(' ٗ', 'S')
        text = text.replace(' ٘', 'D')
        text = text.replace(' ٙ', 'F')
        text = text.replace(' ٚ', 'G')
        text = text.replace(' ٛ', 'H')
        text = text.replace(' ٜ', 'J')
        text = text.replace(' ٝ', 'K')
        text = text.replace(' ٞ', 'L')
        text = text.replace(' ٟ', ':')
        text = text.replace(' ٠', '"')
        text = text.replace(' ١', 'Z')
        text = text.replace(' ٢', 'X')
        text = text.replace(' ٣', 'C')
        text = text.replace(' ٤', 'V')
        text = text.replace(' ٥', 'B')
        text = text.replace(' ٦', 'N')
        text = text.replace(' ٧', 'M')
        text = text.replace(' ٨', '<')
        text = text.replace(' ٩', '>')
        text = text.replace(' ٪', '?')
                
    return text
        
convert("ِاةثي ،اشمي ’آخاشةثي ُسةشهم ُملأشئئشق")

'ِhmed ،hald ’آohamedEsmailElgأazzar'

In [31]:
test = 'qwerwq'
for a in test:
    test[0] = 'Q'
    print(a)

TypeError: 'str' object does not support item assignment

In [16]:
from PyQt5 import QtGui, QtWidgets
from PyQt5.QtWidgets import QFileDialog
from PyQt5.QtWidgets import QApplication
import csv
from collections import namedtuple
import sys
import cv2
import numpy as np


def convert_QI_to_cv(self , incomingImage):
    '''  Converts a QImage into an opencv MAT format  '''

    incomingImage = incomingImage.convertToFormat(4)

    width = incomingImage.width()
    height = incomingImage.height()

    ptr = incomingImage.bits()
    ptr.setsize(incomingImage.byteCount())
    arr = np.array(ptr).reshape(height, width, 4)  #  Copies the data
    return arr

def convert_cv_to_qt(self ,cv_img):
    rgb_image = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
    h, w, ch = rgb_image.shape
    bytes_per_line = ch * w
    convert_to_Qt_format = QtGui.QImage(rgb_image.data, w, h, bytes_per_line, QtGui.QImage.Format_RGB888)
    return convert_to_Qt_format

def draw_bb_id(self ,image, x, y , w, h, id,label, color=(0, 0, 255), thickness=1):
    if self.CURRENT_ANNOATAION_FLAGS['bbox']:
        image = cv2.rectangle(image, (x, y), (x+w, y+h), color, thickness)
        
        
    if self.CURRENT_ANNOATAION_FLAGS['id']  or self.CURRENT_ANNOATAION_FLAGS['class']:
        if self.CURRENT_ANNOATAION_FLAGS['id']  and self.CURRENT_ANNOATAION_FLAGS['class']:
            text = f'#{id} {label}'
        if self.CURRENT_ANNOATAION_FLAGS['id']  and not self.CURRENT_ANNOATAION_FLAGS['class']:
            text = f'#{id}'
        if not self.CURRENT_ANNOATAION_FLAGS['id']  and self.CURRENT_ANNOATAION_FLAGS['class']:
            text = f'{label}'
        
        
        text_width, text_height = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, thickness)[0]
        text_x = x + 10
        text_y = y - 10

        text_background_x1 = x
        text_background_y1 = y - 2 * 10 - text_height

        text_background_x2 = x + 2 * 10 + text_width
        text_background_y2 = y
        cv2.rectangle(
        img=image,
        pt1=(text_background_x1, text_background_y1),
        pt2=(text_background_x2, text_background_y2),
        color=color,
        thickness=cv2.FILLED,
        )
        cv2.putText(
        img=image,
        text=text,
        org=(text_x, text_y),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.7,
        color=(0, 0, 0),
        thickness=thickness,
        lineType=cv2.LINE_AA,
        )    
        

        
    # there is no bbox but there is id or class
    if (not self.CURRENT_ANNOATAION_FLAGS['bbox']) and (self.CURRENT_ANNOATAION_FLAGS['id'] or self.CURRENT_ANNOATAION_FLAGS['class']):
        image = cv2.line(image, (x+int(w/2), y + int(h/2)), (x + 50, y - 12), color, thickness+1)

        
    return image

def draw_bb_on_image(self ,image, shapes):


    img = self.convert_QI_to_cv(image)

    for shape in shapes:
        id = shape["group_id"]
        label = shape["label"]
        
        # color calculation
        idx = coco_classes.index(label)
        idx = idx % len(color_palette)
        color = color_palette[idx]
        
        (x1, y1 , x2, y2) = shape["bbox"]
        x, y , w, h = int(x1), int(y1), int(x2 - x1), int(y2 - y1)
        img = self.draw_bb_id(img, x, y , w, h, id,label, color, thickness = 1)

    qimage = self.convert_cv_to_qt(img)
    return qimage



In [17]:
image = QtGui.QImage('test_img_1.jpg')
detectionData = [(50, 50, 200, 200, 11), (200, 200, 400, 400, 12), (400, 400, 600, 600, 13)]
image = draw_bb_on_image(image, detectionData)


In [24]:
dict = {'group_id': 1, 'label': 'person', 'bbox': (50, 50, 200, 200)}
try:
    x = dict['x']
except:
    dict['x'] = [1,2,3]
    x = None

In [25]:
dict

{'group_id': 1, 'label': 'person', 'bbox': (50, 50, 200, 200), 'x': [1, 2, 3]}

In [ ]:
for i in range(1, len(pts)):
    		# if either of the tracked points are None, ignore
		# them
		if pts[i - 1] is None or pts[i] is None:
			continue
 
		# otherwise, compute the thickness of the line and
		# draw the connecting lines
		thickness = int(np.sqrt(args["buffer"] / float(i + 1)) * 2.5)
		cv2.line(frame, pts[i - 1], pts[i], (0, 0, 255), thickness)

In [ ]:
from PyQt5 import QtGui
from PyQt5.QtWidgets import QApplication, QMainWindow
import sys
from PyQt5.QtGui import QBrush, QPen,QPainter, QPolygon
from PyQt5.QtCore import QPoint, Qt

def draw_bb_on_image(image, x, y , w, h, id, color=(0, 0, 255), thickness=2):
    painter = QPainter()
    painter.setPen(QPen(Qt.black, 5, Qt.SolidLine))
    #painter.setBrush(QBrush(Qt.red, Qt.SolidPattern))
    painter.setBrush(QBrush(Qt.red, Qt.VerPattern))
    points_bb = QPolygon([
                QPoint(x, y),
                QPoint(x, y + h),
                QPoint(x + h, y),
                QPoint(x + h, y + h)
            ])
    points_id = QPolygon([
                QPoint(x, y - 25),
                QPoint(x, y),
                QPoint(x + 100, y - 25),
                QPoint(x + 100, y)
            ])
    painter.drawPolygon(points_bb)
    painter.drawPolygon(points_id)
    